# 第6章: 先見（ステップバイステップで考える）

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル練習場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を設定してください。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## レッスン

誰かがあなたを起こして、すぐにいくつかの複雑な質問を始めて、あなたがすぐに答えなければならないとしたら、どうでしょうか？おそらく、**最初に答えを考える時間を与えられた**場合ほど良い結果は出ないでしょう。

当てられますか？Claude も同じです。

**Claude にステップバイステップで考える時間を与えることで、Claude がより正確になることがあります**。特に複雑なタスクの場合です。しかし、**考えることは声に出してのみカウントされます**。Claude に考えるよう求めて、答えだけを出力してもらうことはできません - この場合、実際には考えることは起こっていません。

### 例

下記のプロンプトでは、人間の読者には2番目の文が最初の文を否定していることが明らかです。しかし、**Claude は「unrelated」という言葉を文字通りに受け取りすぎています**。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

Claude の応答を改善するために、**Claude に答える前にまず物事を考えさせましょう**。Claude がそのタスクを処理し、考え抜くために取るべきステップを文字通り詳しく説明することでそれを行います。ロールプロンプティングと組み合わせることで、Claude はレビューをより深く理解できるようになります。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude は時々順序に敏感です**。この例はClaude の微妙なテキスト理解能力の境界にあり、前の例から議論の順序を入れ替えて、ネガティブを最初に、ポジティブを2番目にすると、Claude の全体的な評価がポジティブに変わります。

ほとんどの状況で（ただし、紛らわしいことに、すべてではありません）、**Claude は2つの選択肢のうち2番目を選ぶ可能性が高くなります**。これは、ウェブからの訓練データで、2番目の選択肢が正しい可能性が高かったためかもしれません。

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**Claude に考えさせることで、Claude の答えを間違いから正解に変えることができます**。Claude がミスをする多くの場合、それはそれほど単純なことです！

Claude の答えが間違っている例を見て、Claude に考えるよう求めることでそれをどう修正できるかを見てみましょう。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Claude にステップバイステップで考えるよう求めて、これを修正しましょう。今回は`<brainstorm>`タグを使います。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

上記のコンテンツを変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**サンプル練習場**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 6.1 - メールの分類](#exercise-61---classifying-emails)
- [演習 6.2 - メール分類の書式設定](#exercise-62---email-classification-formatting)

### 演習 6.1 - メールの分類
この演習では、Claude にメールを以下のカテゴリーに分類するよう指示します：
- (A) 販売前の質問
- (B) 破損または欠陥のあるアイテム
- (C) 請求に関する質問
- (D) その他（説明してください）

演習の最初の部分では、`PROMPT`を変更して、**Claude が正しい分類のみを出力する**ようにしてください。あなたの答えには**正しい選択肢の文字（A - D）を括弧付きで、およびカテゴリの名前を含める**必要があります。

各メールがどのカテゴリーに分類されるべきかを知るために、`EMAILS`リストの各メールの横にあるコメントを参照してください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

まだ行き詰まっていますか？下のセルを実行して解答例を見てください。

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 演習 6.2 - メール分類の書式設定
この演習では、上記のプロンプトの出力を精製して、私たちが望む通りにフォーマットされた答えを得ます。

お気に入りの出力フォーマット技術を使って、Claude に正しい分類の文字のみを`<answer></answer>`タグで囲むようにしてください。例えば、最初のメールへの答えは正確に`<answer>B</answer>`という文字列を含むべきです。

各メールに対してどの文字カテゴリーが正しいかを忘れた場合は、`EMAILS`リストの各メールの横にあるコメントを参照してください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### おめでとうございます！

ここまでのすべての演習を解いた場合、次の章に進む準備ができています。楽しいプロンプティングを！

---

## サンプル練習場

これは、このレッスンで示されたプロンプトの例を自由に試し、プロンプトを調整してClaude の応答にどのような影響を与えるかを確認するためのエリアです。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))